In [1]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from imutils.video import VideoStream
from imutils.video import FPS
import time
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

tf.keras.backend.clear_session()

In [2]:

# Set the directory you want to start from
rootDir = '.'
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        print('\t%s' % fname)
        

# load our serialized face detector from disk
# fix seed for reproducible results (only works on CPU, not GPU)
seed = 1
np.random.seed(seed=seed)
tf.random.set_seed(seed=seed)

# hyper parameters for model
nb_classes = 5  # number of classes
based_model_last_block_layer_number = 126  # value is based on based model selected.
img_width, img_height = 224, 224  # change based on the shape/structure of your images
batch_size = 2  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 100  # number of iteration the algorithm gets trained.
learn_rate = 1e-4  # sgd learning rate
momentum = .9  # sgd momentum to avoid local minimum
transformation_ratio = .0001  # how aggressive will be the data augmentation/transformation
patience = 10

train_data_dir = './dataset_prueba_1/'  # Inside, each class should have it's own folder
#validation_data_dir = './dataset/'  # each class should have it's own folder
model_path = './model/'

Found directory: .
	Train_Classifier_AllDeep.ipynb
	Train_Classifier_DEEP_SVC.ipynb
Found directory: ./.ipynb_checkpoints
	Train_Classifier_AllDeep-checkpoint.ipynb
	Train_Classifier_DEEP_SVC-checkpoint.ipynb
Found directory: ./dataset_prueba_1
Found directory: ./dataset_prueba_1/model_SVM
	embeddings.pickle
	model_SVM.h5
	le.pickle
	recognizer.pickle
Found directory: ./dataset_prueba_1/7- Soiling
	Soling 19 Pampa Camarones M4 2019.jpg
	Soling 6 El Aguila M1 2019.jpg
	Soling 2 El Aguila M1 2019.jpg
	Soling 3 El Aguila M1 2019.jpg
	Soling 15 Pampa Camarones M3 2019.jpg
	Soling 12 Pampa Camarones M2 2019.jpg
	Soling 1 El Aguila M1 2019.jpg
	Soling 20 Pampa Camarones M4 2019.jpg
	Soling 5 El Aguila M1 2019.jpg
	Soling 10 Pampa Camarones M1 2019.jpg
	Soling 9 Pampa Camarones M1 2019.jpg
	Soling 21 Cerro Dominador M1 2019.jpg
	Soling 13 Pampa Camarones M3 2019.jpg
	Soling 4 El Aguila M1 2019.jpg
	Soling 18 Pampa Camarones M4 2019.jpg
	Soling 16 Pampa Camarones M3 2019.jpg
	Soling 7 Pampa Ca

	cat.5561.jpg
	cat.4685.jpg
	cat.2684.jpg
	cat.1667.jpg
	cat.10667.jpg
	cat.124.jpg
	cat.9120.jpg
	cat.2512.jpg
	cat.10994.jpg
	cat.9292.jpg
	cat.9393.jpg
	cat.6116.jpg
	cat.7162.jpg
	cat.4102.jpg
	cat.8426.jpg
	cat.6426.jpg
	cat.6292.jpg
	cat.3952.jpg
	cat.2965.jpg
	cat.5513.jpg
	cat.10067.jpg
	cat.3251.jpg
	cat.5115.jpg
	cat.12080.jpg
	cat.5590.jpg
	cat.3475.jpg
	cat.10608.jpg
	cat.2294.jpg
	cat.6622.jpg
	cat.9110.jpg
	cat.3009.jpg
	cat.1389.jpg
	cat.9532.jpg
	cat.7152.jpg
	cat.7865.jpg
	cat.10513.jpg
	cat.79.jpg
	cat.5055.jpg
	cat.4410.jpg
	cat.9233.jpg
	cat.3647.jpg
	cat.2882.jpg
	cat.9191.jpg
	cat.3357.jpg
	cat.5584.jpg
	cat.9481.jpg
	cat.6578.jpg
	cat.2946.jpg
	cat.9601.jpg
	cat.10977.jpg
	cat.3043.jpg
	cat.7312.jpg
	cat.5702.jpg
	cat.12496.jpg
	cat.3107.jpg
	cat.3487.jpg
	cat.9014.jpg
	cat.5148.jpg
	cat.7099.jpg
	cat.10091.jpg
	cat.10448.jpg
	cat.4849.jpg
	cat.7664.jpg
	cat.4587.jpg
	cat.8918.jpg
	cat.8205.jpg
	cat.6883.jpg
	cat.8579.jpg
	cat.1789.jpg
	cat.8438.jpg
	cat.2244.jpg

	cat.10063.jpg
	cat.556.jpg
	cat.12097.jpg
	cat.4310.jpg
	cat.7655.jpg
	cat.4017.jpg
	cat.1374.jpg
	cat.3207.jpg
	cat.75.jpg
	cat.3835.jpg
	cat.11212.jpg
	cat.3816.jpg
	cat.5789.jpg
	cat.11594.jpg
	cat.4253.jpg
	cat.4433.jpg
	cat.4902.jpg
	cat.11863.jpg
	cat.3419.jpg
	cat.2732.jpg
	cat.11322.jpg
	cat.10192.jpg
	cat.6106.jpg
	cat.10078.jpg
	cat.722.jpg
	cat.7959.jpg
	cat.8042.jpg
	cat.8860.jpg
	cat.4870.jpg
	cat.7135.jpg
	cat.6899.jpg
	cat.3278.jpg
	cat.2135.jpg
	cat.391.jpg
	cat.3790.jpg
	cat.1986.jpg
	cat.6629.jpg
	cat.6326.jpg
	cat.1712.jpg
	cat.2524.jpg
	cat.8919.jpg
	cat.4022.jpg
	cat.998.jpg
	cat.11944.jpg
	cat.11518.jpg
	cat.10369.jpg
	cat.10345.jpg
	cat.7908.jpg
	cat.1753.jpg
	cat.2983.jpg
	cat.2334.jpg
	cat.960.jpg
	cat.10756.jpg
	cat.7247.jpg
	cat.3907.jpg
	cat.3121.jpg
	cat.12198.jpg
	cat.6053.jpg
	cat.4209.jpg
	cat.1515.jpg
	cat.845.jpg
	cat.11441.jpg
	cat.1023.jpg
	cat.43.jpg
	cat.1591.jpg
	cat.10227.jpg
	cat.64.jpg
	cat.3111.jpg
	cat.12041.jpg
	cat.4165.jpg
	cat.875.jpg
	c

	cat.8588.jpg
	cat.6744.jpg
	cat.9460.jpg
	cat.6698.jpg
	cat.824.jpg
	cat.9933.jpg
	cat.10776.jpg
	cat.1268.jpg
	cat.8459.jpg
	cat.7715.jpg
	cat.8169.jpg
	cat.9648.jpg
	cat.11086.jpg
	cat.4019.jpg
	cat.5951.jpg
	cat.11161.jpg
	cat.3247.jpg
	cat.567.jpg
	cat.182.jpg
	cat.12199.jpg
	cat.8583.jpg
	cat.9696.jpg
	cat.10084.jpg
	cat.4967.jpg
	cat.10705.jpg
	cat.6023.jpg
	cat.2338.jpg
	cat.6517.jpg
	cat.3500.jpg
	cat.10861.jpg
	cat.10769.jpg
	cat.2956.jpg
	cat.3812.jpg
	cat.3416.jpg
	cat.9366.jpg
	cat.4153.jpg
	cat.8098.jpg
	cat.2116.jpg
	cat.314.jpg
	cat.1330.jpg
	cat.10163.jpg
	cat.3301.jpg
	cat.4081.jpg
	cat.4386.jpg
	cat.7754.jpg
	cat.8133.jpg
	cat.1255.jpg
	cat.9604.jpg
	cat.4005.jpg
	cat.2417.jpg
	cat.3691.jpg
	cat.224.jpg
	cat.194.jpg
	cat.10731.jpg
	cat.3445.jpg
	cat.10297.jpg
	cat.10074.jpg
	cat.11131.jpg
	cat.4380.jpg
	cat.965.jpg
	cat.461.jpg
	cat.833.jpg
	cat.5875.jpg
	cat.9873.jpg
	cat.11330.jpg
	cat.8805.jpg
	cat.10567.jpg
	cat.11733.jpg
	cat.2834.jpg
	cat.12409.jpg
	cat.7210.jp

	dog.5276.jpg
	dog.3329.jpg
	dog.7893.jpg
	dog.3585.jpg
	dog.7650.jpg
	dog.1290.jpg
	dog.3142.jpg
	dog.11376.jpg
	dog.2491.jpg
	dog.6638.jpg
	dog.6429.jpg
	dog.2626.jpg
	dog.8878.jpg
	dog.475.jpg
	dog.89.jpg
	dog.8506.jpg
	dog.9247.jpg
	dog.4089.jpg
	dog.5284.jpg
	dog.8001.jpg
	dog.20.jpg
	dog.6180.jpg
	dog.4494.jpg
	dog.9948.jpg
	dog.11548.jpg
	dog.12364.jpg
	dog.5808.jpg
	dog.8499.jpg
	dog.12059.jpg
	dog.5891.jpg
	dog.4300.jpg
	dog.3593.jpg
	dog.2404.jpg
	dog.3875.jpg
	dog.5125.jpg
	dog.8551.jpg
	dog.12148.jpg
	dog.3662.jpg
	dog.653.jpg
	dog.10520.jpg
	dog.9378.jpg
	dog.3120.jpg
	dog.3577.jpg
	dog.2079.jpg
	dog.10287.jpg
	dog.3652.jpg
	dog.5552.jpg
	dog.7223.jpg
	dog.7191.jpg
	dog.3400.jpg
	dog.12245.jpg
	dog.68.jpg
	dog.10084.jpg
	dog.10016.jpg
	dog.9359.jpg
	dog.4581.jpg
	dog.4458.jpg
	dog.8834.jpg
	dog.7127.jpg
	dog.12350.jpg
	dog.11409.jpg
	dog.8222.jpg
	dog.3994.jpg
	dog.2200.jpg
	dog.1129.jpg
	dog.5918.jpg
	dog.9260.jpg
	dog.985.jpg
	dog.11747.jpg
	dog.12074.jpg
	dog.7877.jpg
	

	dog.6346.jpg
	dog.8074.jpg
	dog.4857.jpg
	dog.6423.jpg
	dog.7760.jpg
	dog.4504.jpg
	dog.6190.jpg
	dog.5266.jpg
	dog.8424.jpg
	dog.9084.jpg
	dog.4684.jpg
	dog.5572.jpg
	dog.585.jpg
	dog.8629.jpg
	dog.3627.jpg
	dog.5597.jpg
	dog.8755.jpg
	dog.11036.jpg
	dog.3085.jpg
	dog.11248.jpg
	dog.9631.jpg
	dog.1417.jpg
	dog.12419.jpg
	dog.6956.jpg
	dog.9918.jpg
	dog.9389.jpg
	dog.892.jpg
	dog.8456.jpg
	dog.8452.jpg
	dog.1485.jpg
	dog.7239.jpg
	dog.3814.jpg
	dog.6568.jpg
	dog.10873.jpg
	dog.7380.jpg
	dog.3315.jpg
	dog.2825.jpg
	dog.10030.jpg
	dog.5661.jpg
	dog.11977.jpg
	dog.5607.jpg
	dog.8136.jpg
	dog.1049.jpg
	dog.11190.jpg
	dog.2361.jpg
	dog.699.jpg
	dog.9749.jpg
	dog.8094.jpg
	dog.5259.jpg
	dog.2096.jpg
	dog.5625.jpg
	dog.7122.jpg
	dog.5583.jpg
	dog.10790.jpg
	dog.6813.jpg
	dog.9533.jpg
	dog.10895.jpg
	dog.3744.jpg
	dog.3752.jpg
	dog.10678.jpg
	dog.9070.jpg
	dog.9510.jpg
	dog.4013.jpg
	dog.9090.jpg
	dog.10437.jpg
	dog.9141.jpg
	dog.10830.jpg
	dog.6708.jpg
	dog.7790.jpg
	dog.11687.jpg
	dog.4651.

	dog.1007.jpg
	dog.11920.jpg
	dog.2481.jpg
	dog.8221.jpg
	dog.12281.jpg
	dog.10605.jpg
	dog.11611.jpg
	dog.5755.jpg
	dog.3327.jpg
	dog.10211.jpg
	dog.3322.jpg
	dog.4155.jpg
	dog.11627.jpg
	dog.6633.jpg
	dog.283.jpg
	dog.12269.jpg
	dog.10547.jpg
	dog.2563.jpg
	dog.1449.jpg
	dog.1598.jpg
	dog.8260.jpg
	dog.5767.jpg
	dog.1229.jpg
	dog.1843.jpg
	dog.9873.jpg
	dog.12437.jpg
	dog.9934.jpg
	dog.9327.jpg
	dog.11103.jpg
	dog.8140.jpg
	dog.7096.jpg
	dog.6143.jpg
	dog.10309.jpg
	dog.6791.jpg
	dog.8430.jpg
	dog.10809.jpg
	dog.9651.jpg
	dog.7515.jpg
	dog.5791.jpg
	dog.10483.jpg
	dog.4157.jpg
	dog.7940.jpg
	dog.10266.jpg
	dog.3392.jpg
	dog.12410.jpg
	dog.9865.jpg
	dog.3812.jpg
	dog.2537.jpg
	dog.12248.jpg
	dog.9405.jpg
	dog.5598.jpg
	dog.4226.jpg
	dog.1403.jpg
	dog.11318.jpg
	dog.6207.jpg
	dog.8089.jpg
	dog.4201.jpg
	dog.4370.jpg
	dog.9556.jpg
	dog.216.jpg
	dog.10825.jpg
	dog.6619.jpg
	dog.9482.jpg
	dog.7132.jpg
	dog.10376.jpg
	dog.10990.jpg
	dog.3026.jpg
	dog.609.jpg
	dog.9596.jpg
	dog.7296.jpg
	do

Found directory: ./model
	model.json
	top_model_weights.h5
	model_weights.h5
Found directory: ./preview


In [3]:


base_model = tf.keras.applications.Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
#base_model = tf.keras.applications.MobileNetV2(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
#base_model = tf.keras.applications.NASNetMobile(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)


# Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(nb_classes, activation='softmax')(x)

# add your top layer block to your base model
model = tf.keras.Model(base_model.input, predictions)


# # let's visualize layer names and layer indices to see how many layers/blocks to re-train
# # uncomment when choosing based_model_last_block_layer
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all layers of the based model that is already pre-trained.
for layer in base_model.layers:
    layer.trainable = True

print(model.summary())
# Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
# To save augmentations un-comment save lines and add to your flow parameters.
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=transformation_ratio,
                                   shear_range=transformation_ratio,
                                   zoom_range=transformation_ratio,
                                   cval=transformation_ratio,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)



train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(train_data_dir, # same directory as training data
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        class_mode='binary',
                                                        subset='validation') # set as validation data

model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
              metrics=['accuracy'])

# save weights of best training epoch: monitor either val_loss or val_acc

top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience= patience, verbose=0)
]

# Train Simple CNN
hist = model.fit(train_generator,
                    steps_per_epoch= int(train_generator.samples/batch_size*.8),
                    epochs=int (nb_epoch / 5),
                    validation_data=validation_generator,
                    validation_steps = int(validation_generator.samples /batch_size*.8),
                    validation_freq=1,
                    callbacks=callbacks_list)



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 58 steps, validate for 13 steps
Epoch 1/20
57/58 [============================>.] - ETA: 0s - loss: 15.7914 - accuracy: 0.0614
Epoch 00001: val_loss improved from inf to 16.76773, saving model to /home/dlsaavedra/Desktop/Rentadrone.cl-ai-test&SomeCode/model-definition/GPS_Panel/Classifier/model/top_model_weights.h5
58/58 [==============================] - 50s 862ms/step - loss: 15.5191 - accuracy: 0.0603 - val_loss: 16.7677 - val_accuracy: 0.0000e+00
Epoch 2/20
57/58 [============================>.] - ETA: 0s - loss: 15.5472 - accuracy: 0.0088
Epoch 00002: val_loss did not improve from 16.76773
58/58 [==============================] - 49s 837ms/step - loss: 15.7657 - accuracy: 0.0086 - val_loss: 22.9064 - val_accuracy: 0.0000e+00
Epoch 3/20
57/58 [============================>.] - ETA: 0s - loss: 15.3652 - accuracy: 0.0175
Epoch 00003: val_loss did not improve from 16.76773
58/58 [==============================] - 50s 871ms/st

In [4]:
# verbose
print("\nStarting to Fine Tune Model\n")

# add the best weights from the train top model
# at this point we have the pre-train weights of the base model and the trained weight of the new/added top model
# we re-load model weights to ensure the best epoch is selected and not the last one.
model.load_weights(top_weights_path)

print(model.summary())
# based_model_last_block_layer_number points to the layer in your model you want to train.
# For example if you want to train the last block of a 19 layer VGG16 model this should be 15
# If you want to train the last Two blocks of an Inception model it should be 172
# layers before this number will used the pre-trained weights, layers above and including this number
# will be re-trained based on the new data.
for layer in model.layers[:based_model_last_block_layer_number]:
    layer.trainable = True
for layer in model.layers[based_model_last_block_layer_number:]:
    layer.trainable = True

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# save weights of best training epoch: monitor either val_loss or val_acc
final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=patience, verbose=0)
]

# fine-tune the model
hist = model.fit(train_generator,
            steps_per_epoch=train_generator.samples/batch_size * .8,
            epochs=nb_epoch,
            validation_data=validation_generator,
            validation_steps = validation_generator.samples/batch_size*.8,
            callbacks=callbacks_list)

# save model
model_json = model.to_json()
with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
    json_file.write(model_json)


Starting to Fine Tune Model

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
________________________________________________________________

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 58.400000000000006 steps, validate for 13.600000000000001 steps
Epoch 1/100
58/58 [============================>.] - ETA: 0s - loss: 16.7909 - accuracy: 0.1207
Epoch 00001: val_loss improved from inf to 17.16391, saving model to /home/dlsaavedra/Desktop/Rentadrone.cl-ai-test&SomeCode/model-definition/GPS_Panel/Classifier/model/model_weights.h5
59/58 [==============================] - 75s 1s/step - loss: 16.5745 - accuracy: 0.1186 - val_loss: 17.1639 - val_accuracy: 0.4286
Epoch 2/100
58/58 [============================>.] - ETA: 0s - loss: 16.1648 - accuracy: 0.0948
Epoch 00002: val_loss did not improve from 17.16391
59/58 [==============================] - 56s 948ms/step - loss: 16.1636 - accuracy: 0.0932 - val_loss: 17.1654 - val_accuracy: 0.1429
Epoch 3/100
58/58 [============================>.] - ETA: 0s - loss: 15.5401 - accuracy: 0.0690
Epoch 00003: val_loss did not improve from 17.16391
59/58 [==========================

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

json_file = open(os.path.join(os.path.abspath(model_path), 'model.json'), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)


#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.
Confusion Matrix
[[0 4 1 0 0 0]
 [0 7 5 1 0 0]
 [0 2 1 1 1 0]
 [0 1 3 0 0 0]
 [0 2 0 0 2 0]
 [0 2 0 1 0 0]]
Classification Report


ValueError: Number of classes, 6, does not match size of target_names, 7. Try specifying the labels parameter